In [1]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [2]:
root = "../raw/it.openfoodfacts.org"
foods = [food for food in os.listdir(root) if food[0] != "."]

### Merging csv files

In [22]:
def parse_food(food):
    root = "../raw/it.openfoodfacts.org"
    file_path = "{path}/{name}/Informazioni nutrizionali.csv".format(
        path=root, name=food)
    try:
        df = pd.read_csv(
            file_path,
            usecols=[
                "Informazioni nutrizionali", "Come vendutoper 100 g / 100 ml"
            ],
            index_col="Informazioni nutrizionali")
    except Exception as e:
        return None
    if (df == "?").any().any():
        return None
    for index, value in df.iterrows():
        value = value["Come vendutoper 100 g / 100 ml"]
        if isinstance(value, str):
            unit = ""
            value = value.replace(".", "").replace(",", ".").replace("<", "")
            if "mg" in value:
                value = float(value.split("mg")[0])*1e-3
                unit = "g"
            elif "µg" in value:
                value = float(value.split("µg")[0])*1e-6
                unit = "g"
            elif "g" in value:
                value = float(value.split("g")[0])
                unit = "g"
            elif re.match("-?[\dA-Z]", value):
                pass
            else:
                raise ValueError(str((value, index)))
            df.loc[index] = value
            if unit:
                df = df.rename(
                    index={
                        index:
                        "{index} | {unit}".format(index=index, unit=unit).lower()
                        .strip().replace("  ", " ")
                    })

    df = df.transpose()
    with open("{path}/{name}/metadata.json".format(path=root, name=food),
              "r") as f:
        for key, value in json.load(f).items():
            if value:
                df[key] = value
    df.index = df["name"]
    df.columns = [c.replace("-", "").strip() for c in df.columns]
    del df.columns.name
    return df

In [28]:
from IPython.display import display
with Pool(cpu_count()) as p:
    open_food_facts = pd.concat(
        list(tqdm(p.imap(parse_food, foods), total=len(foods))))
open_food_facts.index.name = "name"

In [47]:
open_food_facts.to_csv("../csv/openfoodsfacts.csv")